# OM 620 — Milestone III: Inventory Analytics & Safety Stock Showcase
**Student:** Brenda Laime  
**Course:** OM 620 — Tools & Technologies for Analytics  
**Dataset:** `transaction_data.csv` (same dataset as Assignments 1 & 2)

This single notebook **combines Assignment 1 and Assignment 2** to showcase the full workflow:

- **Assignment 1 (Data Cleaning & Filtering)**  
  1) Standardize/clean columns and types  
  2) Inspect numeric features, fix anomalies (returns, invalid prices)  
  3) Handle missing values  
  4) Filter to FG + MTS and answer basic questions

- **Assignment 2 (Safety Stock)**  
  1) Transform to per-SKU stats (min/max/avg/median/var/std, avg lead time)  
  2) Compute safety stock at 75%, 90%, 95% service levels  
  3) Report the largest/smallest/average safety stock (95%)  
  4) **Side Quest**: Empirical (non-parametric) safety stock using 95th percentile

> This notebook is **self-contained**: it reloads and cleans the raw data so it can be run independently. 


## Assignment 1 — Data Cleaning & Filtering

In [ ]:
import pandas as pd
import numpy as np

# Load data (place this notebook under repo/notebooks and put CSV in repo/data)
df = pd.read_csv("../data/transaction_data.csv")

# --- Q1: Dataframe formatting (standardize columns) ---
df.columns = (
    df.columns
    .str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("-", "_")
)
df.rename(columns={"sku_numb3r": "sku_number"}, inplace=True)

# --- Q2: Inspection & anomaly checks ---
df['transaction_date'] = pd.to_datetime(df['transaction_date'], errors='coerce')

# returns flag for negative quantities
df['is_return'] = df['order_quantity'] < 0

# non-positive unit prices -> mark missing (then we fill)
df.loc[df['unit_price'] <= 0, 'unit_price'] = np.nan

print('Negative order quantities:', (df['order_quantity'] < 0).sum())
print('Non-positive unit prices (now NaN):', df['unit_price'].isna().sum())

# --- Q3: Handle NaNs ---
fill_map = {
    "order_quantity": 0,
    "unit_price": 0,
    "sku_number": "Unknown",
    "inventory_type": "Unknown",
    "stocking_type": "Unknown",
    "manufacturing_site": "Unknown",
    "division_code": "Unknown",
}
df = df.fillna(fill_map)

# --- Q4: Useful information on FG + MTS subset ---
fg_mts = df[(df["inventory_type"] == "FG") & (df["stocking_type"] == "MTS")].copy()

# Add total_sales for reporting
fg_mts['total_sales'] = fg_mts['order_quantity'] * fg_mts['unit_price']

# Summary answers
unique_skus = fg_mts['sku_number'].nunique()
unique_sites = fg_mts['manufacturing_site'].nunique()
unique_divs = fg_mts['division_code'].nunique()

print('Unique SKUs:', unique_skus)
print('Unique Manufacturing Sites:', unique_sites)
print('Unique Division Codes:', unique_divs)

# Preview top/bottom examples
cols_show = ["sku_number","manufacturing_site","division_code","order_quantity","unit_price","total_sales"]

top10_qty = fg_mts.sort_values('order_quantity', ascending=False).head(10)[cols_show]
bottom10_qty = fg_mts.sort_values('order_quantity', ascending=True).head(10)[cols_show]

top10_sales = fg_mts.sort_values('total_sales', ascending=False).head(10)[cols_show]
bottom10_sales = fg_mts.sort_values('total_sales', ascending=True).head(10)[cols_show]

top10_qty.head(3), bottom10_qty.head(3), top10_sales.head(3), bottom10_sales.head(3)

**A1 notes (short):**  
- I keep negative `order_quantity` as **returns** via `is_return`.  
- I set invalid `unit_price` (≤ 0) to `NaN` and then fill with 0 (so later math runs).  
- I filled missing text with `"Unknown"` to retain rows.  
- Then I filtered to **Finished Goods + Make-To-Stock (FG + MTS)** for all downstream analysis.  
- I added `total_sales = order_quantity × unit_price` for interpretable top/bottom tables.


## Assignment 2 — Safety Stock (Service Levels & Empirical)

### Q1 — Per-SKU Transformation
For each SKU in the FG+MTS subset, compute: min, max, mean, median, variance, standard deviation of `order_quantity`, and average `lead_time`. 


In [ ]:
sku_stats = (
    fg_mts.groupby("sku_number")
    .agg(
        min_qty = ("order_quantity", "min"),
        max_qty = ("order_quantity", "max"),
        avg_qty = ("order_quantity", "mean"),
        median_qty = ("order_quantity", "median"),
        var_qty = ("order_quantity", "var"),
        std_qty = ("order_quantity", "std"),
        avg_lead_time = ("lead_time", "mean"),
    )
    .reset_index()
)
sku_stats.head()

### Q2 — Safety Stock @ 75%, 90%, 95%
Formula: **SS = z × σ × √L**  
- 75% → z ≈ 0.674  
- 90% → z ≈ 1.282  
- 95% → z ≈ 1.645  
Where σ is `std_qty` and L is `avg_lead_time`.


In [ ]:
z_75, z_90, z_95 = 0.674, 1.282, 1.645
sku_stats['sqrt_lead_time'] = np.sqrt(sku_stats['avg_lead_time'])

sku_stats['safety_stock_75'] = z_75 * sku_stats['std_qty'] * sku_stats['sqrt_lead_time']
sku_stats['safety_stock_90'] = z_90 * sku_stats['std_qty'] * sku_stats['sqrt_lead_time']
sku_stats['safety_stock_95'] = z_95 * sku_stats['std_qty'] * sku_stats['sqrt_lead_time']

sku_stats[['safety_stock_75','safety_stock_90','safety_stock_95']] = (
    sku_stats[['safety_stock_75','safety_stock_90','safety_stock_95']].fillna(0)
)
sku_stats.head()

### Q3 — Largest / Smallest / Average (95% Service Level)


In [ ]:
safety_95 = sku_stats[['sku_number','safety_stock_95']].copy()
max_idx = safety_95['safety_stock_95'].idxmax()
min_idx = safety_95['safety_stock_95'].idxmin()
max_row = safety_95.loc[max_idx]
min_row = safety_95.loc[min_idx]
avg_safety_95 = safety_95['safety_stock_95'].mean()

print('Highest safety stock SKU (95% level):')
print(f"  SKU: {max_row['sku_number']}  |  Safety Stock: {max_row['safety_stock_95']:.2f}")
print('\nLowest safety stock SKU (95% level):')
print(f"  SKU: {min_row['sku_number']}  |  Safety Stock: {min_row['safety_stock_95']:.2f}")
print('\nAverage safety stock across all SKUs (95% level):')
print(f"  {avg_safety_95:.2f}")

max_row, min_row, avg_safety_95

### Side Quest — Empirical (Non‑Parametric) Safety Stock (95th Percentile)
Use **observed** demand (no normality assumption):  
`SS_empirical_95 = quantile_95(order_quantity) × √(avg_lead_time)`


In [ ]:
quantile_stats = (
    fg_mts.groupby('sku_number')
    .agg(
        q95_demand = ('order_quantity', lambda x: np.quantile(x, 0.95)),
        avg_lead_time = ('lead_time', 'mean')
    )
    .reset_index()
)

quantile_stats['safety_stock_empirical_95'] = (
    quantile_stats['q95_demand'] * np.sqrt(quantile_stats['avg_lead_time'])
)
quantile_stats.head()

#### Quick Comparison (normal vs empirical)
This table helps spot SKUs where the **normal-based** and **empirical** methods diverge the most.


In [ ]:
compare = (
    quantile_stats[['sku_number','safety_stock_empirical_95']]
    .merge(sku_stats[['sku_number','safety_stock_95']], on='sku_number', how='inner')
    .assign(diff=lambda d: d['safety_stock_empirical_95'] - d['safety_stock_95'])
    .sort_values('diff', ascending=False)
)

# Show top 5 where empirical >> normal
top5_empirical_over = compare.head(5)

# Show top 5 where normal >> empirical
top5_normal_over = compare.sort_values('diff').head(5)

top5_empirical_over, top5_normal_over

## Conclusion (Milestone III)
- Combined **cleaning & filtering** (A1) with **safety stock** (A2) in a single, reproducible notebook.
- Cleaned dataset (consistent columns, dates, returns flagged, NaNs handled).  
- Focused on **FG + MTS** SKUs and reported core counts and top/bottom tables.
- Created per‑SKU demand/lead time summaries and computed safety stock at 75/90/95% service levels.  
- Answered the 95% service level questions (largest, smallest, average).
- The **Side Quest** used an empirical 95th percentile method (no normality assumption) and included a quick comparison to flag SKUs where the normal formula might over/under-estimate.


## References
- OM 620 class notes and slides (Dr. Majid Karimi)  
- Pandas documentation (groupby, agg, quantile)  
- NumPy documentation  
- Collaboration/discussion with my classmate Chris  
